In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="qzy6wADsZuZjjvruIPY7")
project = rf.workspace("heithem-benmoussa-hvbkv").project("my-first-projectclas")
version = project.version(1)
dataset = version.download("folder")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 10.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to My-First-Projectclas-1 in folder:: 100%|██████████| 5430/5430 [00:02<00:00, 2646.12it/s]


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import json

In [ ]:
from torchvision import transforms
import torch
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
data_dir = "/content/My-First-Projectclas-1"
test_dir = os.path.join(data_dir, "test")
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "valid")

transform = {
    'train': transforms.Compose([
        transforms.Resize(256),  
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Augmentation de couleurs
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalisation pour EfficientNet-B0
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),  
        transforms.CenterCrop(224),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalisation pour EfficientNet-B0
    ]),
    'test': transforms.Compose([  
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}


In [ ]:

train_dataset = datasets.ImageFolder(train_dir, transform=transform['train'])
val_dataset = datasets.ImageFolder(val_dir, transform=transform['val'])
test_dataset = datasets.ImageFolder(test_dir, transform=transform['test'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
model = models.efficientnet_b0(pretrained=True)
num_classes = len(train_dataset.classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.


In [ ]:

class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0, path='best_model.pth'):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping: Pas d'amélioration ({self.counter}/{self.patience})")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f"Validation Loss améliorée ({self.val_loss_min:.6f} → {val_loss:.6f}). Sauvegarde du modèle...")
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


In [ ]:
def calculate_metrics(y_true, y_pred, num_classes):
    precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
    recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
    cm = confusion_matrix(y_true, y_pred, labels=list(range(num_classes)))
    return precision, recall, f1, cm


In [ ]:
early_stopping = EarlyStopping(patience=10, verbose=True, path='best_model.pth')

num_epochs = 50
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct / total

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    precision, recall, f1, cm = calculate_metrics(all_labels, all_preds, num_classes)
    val_loss /= len(val_loader)
    val_acc = 100. * val_correct / val_total

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}\n")

    # Early stopping check
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Arrêt anticipé !")
        break


Epoch 1/50
Train Loss: 0.2094, Train Acc: 91.73%
Val Loss: 0.6553, Val Acc: 79.17%
Precision: 0.7960, Recall: 0.7949, F1 Score: 0.7916

Validation Loss améliorée (inf → 0.655282). Sauvegarde du modèle...
Epoch 2/50
Train Loss: 0.1089, Train Acc: 97.09%
Val Loss: 0.8010, Val Acc: 81.94%
Precision: 0.8218, Recall: 0.8166, F1 Score: 0.8177

EarlyStopping: Pas d'amélioration (1/10)
Epoch 3/50
Train Loss: 0.0688, Train Acc: 97.73%
Val Loss: 0.6952, Val Acc: 77.78%
Precision: 0.7810, Recall: 0.7740, F1 Score: 0.7750

EarlyStopping: Pas d'amélioration (2/10)
Epoch 4/50
Train Loss: 0.0401, Train Acc: 99.14%
Val Loss: 1.1746, Val Acc: 81.94%
Precision: 0.8205, Recall: 0.8212, F1 Score: 0.8194

EarlyStopping: Pas d'amélioration (3/10)
Epoch 5/50
Train Loss: 0.0938, Train Acc: 97.07%
Val Loss: 0.4983, Val Acc: 79.17%
Precision: 0.7975, Recall: 0.7872, F1 Score: 0.7884

Validation Loss améliorée (0.655282 → 0.498339). Sauvegarde du modèle...
Epoch 6/50
Train Loss: 0.0468, Train Acc: 98.32%
Val Los

In [ ]:

model.load_state_dict(torch.load('best_model.pth'))
print("Modèle chargé avec les meilleurs poids !")

model.eval()
test_correct = 0
test_total = 0
all_test_labels = []
all_test_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)

        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

        all_test_labels.extend(labels.cpu().numpy())
        all_test_preds.extend(predicted.cpu().numpy())

precision_test, recall_test, f1_test, cm_test = calculate_metrics(all_test_labels, all_test_preds, num_classes)
test_acc = 100. * test_correct / test_total

print(f"Test Accuracy: {test_acc:.2f}%")
print(f"Test Precision: {precision_test:.4f}, Recall: {recall_test:.4f}, F1 Score: {f1_test:.4f}")
print(f"Confusion Matrix:\n{cm_test}")



In [ ]:
metrics = {
    "Validation": {
        "Loss": val_loss,
        "Accuracy": val_acc,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    },
    "Test": {
        "Accuracy": test_acc,
        "Precision": precision_test,
        "Recall": recall_test,
        "F1 Score": f1_test,
        "Confusion Matrix": cm_test.tolist()
    }
}

with open("metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)
print("Métriques sauvegardées dans metrics.json")

Métriques sauvegardées dans metrics.json
